In [2]:
import cv2
import os
import numpy as np
from collections import defaultdict, deque

In [3]:
DATASET_DIR="../imc_25/image-matching-challenge-2025/train/imc2023_haiper"
INLIER_THRESHOLD = 15

image_names = sorted(os.listdir(DATASET_DIR))
image_names=image_names[::7]  # Subsample for faster processing
num_images = len(image_names)


In [8]:
image_names

['bike_image_004.png',
 'bike_image_094.png',
 'bike_image_150.png',
 'chairs_image_094.png',
 'chairs_image_152.png',
 'fountain_image_033.png',
 'fountain_image_116.png',
 'fountain_image_173.png']

In [4]:
images = {}
for name in image_names:
    path = os.path.join(DATASET_DIR, name)
    images[name] = cv2.imread(path, cv2.IMREAD_GRAYSCALE)


In [5]:
sift = cv2.SIFT_create()

keypoints = {}
descriptors = {}

for name, img in images.items():
    kp, des = sift.detectAndCompute(img, None)
    keypoints[name] = kp
    descriptors[name] = des

print("Feature extraction done.")


Feature extraction done.


In [6]:
import tqdm

bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

graph = defaultdict(set)

for i in tqdm.tqdm(range(num_images)):
    for j in range(i + 1, num_images):
        name1 = image_names[i]
        name2 = image_names[j]

        des1 = descriptors[name1]
        des2 = descriptors[name2]

        if des1 is None or des2 is None:
            continue

        matches = bf.match(des1, des2)
        if len(matches) < 8:
            continue

        kp1 = keypoints[name1]
        kp2 = keypoints[name2]

        pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
        pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

        F, mask = cv2.findFundamentalMat(
            pts1, pts2,
            cv2.FM_RANSAC,
            ransacReprojThreshold=1.0,
            confidence=0.99
        )

        inliers = 0 if mask is None else int(mask.sum())

        if inliers >= INLIER_THRESHOLD:
            graph[name1].add(name2)
            graph[name2].add(name1)

print("Graph construction done.")


100%|██████████| 8/8 [11:33<00:00, 86.68s/it] 

Graph construction done.


In [7]:
visited = set()
components = []

for name in image_names:
    if name in visited:
        continue

    queue = deque([name])
    visited.add(name)
    component = [name]

    while queue:
        current = queue.popleft()
        for neighbor in graph[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                component.append(neighbor)

    components.append(component)

print("Connected components:")
for idx, comp in enumerate(components):
    print(f"Component {idx}: size = {len(comp)}")


Connected components:
Component 0: size = 8
